### Architetture dati - progetto use case "Data management for Machine Learning"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score

from sklearn.tree import DecisionTreeClassifier

import time

## 1. Inizializzazione del dataset load_breast_cancer()
Il dataset è ottimo per questo progetto dato il numero elavato di features (30).


Il dataset contiene 30 caratteristiche (attributi) che descrivono le proprietà dei nuclei delle cellule tumorali ottenute tramite immagini di aspirato con ago sottile (FNA) di una massa al seno. Le caratteristiche sono suddivise in tre gruppi principali: media, errore standard, e i "peggiori" (cioè, il valore massimo) di queste caratteristiche per ogni immagine.


Il dataset include anche una variabile di destinazione (target) binaria che indica se il tumore è maligno (0) o benigno (1).

Inoltre questo dataset risulta comodo in quanto disponibile direttamente da sklearn.datasets

In [ ]:
data = pd.read_csv('./Dataset/breast-cancer-augmented.csv')
data.head()
X = data.drop(columns=['diagnosis'])
Y = data['diagnosis'].map({'M': 0 , 'B': 1})

df = pd.DataFrame(X)
df['target'] = Y
df.drop(columns=['id'], inplace=True)

df.head()


## 2. Analisi esplorativa dei dati
Vado adesso ad eseguire un analisi esplorativa dei dati per capire meglio il dataset.

### 2.1 Verifica del numero di features e di istanze
Come prima cosa verifichiamo il numero di features e di istanze: 30 e 569 rispettivamente.


Il target è binario:
* 0 = Maligno
* 1 = Benigno

In [ ]:
#print number of features
print(f"Number of features: {len(data.columns)}")
#print number of istances
print(f"Number of istances: {len(df)}")

df.head()

### 2.2 Distribuzione della Diagnosi del Tumore

Questo codice genera un grafico a barre che rappresenta la distribuzione della variabile target "target" nel dataset "df". La variabile target indica la diagnosi del tumore, con 0 che indica un tumore maligno e 1 che indica un tumore benigno.

Il grafico è suddiviso in due barre:

* **Barra rossa:** Rappresenta la frequenza dei tumori maligni.
* **Barra blu:** Rappresenta la frequenza dei tumori benigni.

L'asse x del grafico rappresenta la diagnosi del tumore (0 = maligno, 1 = benigno). L'asse y del grafico rappresenta la frequenza delle diverse diagnosi.

**Interpretazione:**

Il grafico mostra che nel dataset "Breast Cancer Wisconsin ..." ci sono più tumori benigni che tumori maligni. La frequenza dei tumori maligni è circa il 37.2%, mentre la frequenza dei tumori benigni è circa il 62.7%.

**Informazioni aggiuntive:**

* Il codice utilizza la libreria seaborn per generare il grafico a barre.
* La funzione `sns.countplot()` è utilizzata per creare il grafico a barre.
* L'argomento `data=df` indica il dataset da utilizzare per generare il grafico.
* L'argomento `x='target'` indica la variabile da utilizzare per l'asse x del grafico.
* L'argomento `palette=['red', 'blue']` indica i colori da utilizzare per le barre del grafico.
* L'argomento `hue='target'` indica che le barre del grafico devono essere suddivise in base al valore della variabile target.
* L'argomento `dodge=False` indica che le barre del grafico non devono essere separate.
* L'argomento `legend=False` indica che la legenda del grafico non deve essere visualizzata.
* Il metodo `plt.title()` è utilizzato per impostare il titolo del grafico.
* Il metodo `plt.xlabel()` è utilizzato per impostare l'etichetta dell'asse x del grafico.
* Il metodo `plt.ylabel()` è utilizzato per impostare l'etichetta dell'asse y del grafico.
* Il metodo `plt.show()` è utilizzato per visualizzare il grafico.

In [ ]:
# Ottenere il conteggio e le percentuali per la variabile "target"
conteggio_percentuali = df['target'].value_counts(normalize=True) * 100

# Stampare le percentuali
print("Percentuali di diagnosi:")
print(conteggio_percentuali)


plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='target', palette=['red', 'blue'], hue='target', dodge=False, legend=False)
plt.title('Distribuzione della Diagnosi del Tumore')
plt.xlabel('Diagnosi (0 = Maligno, 1 = Benigno)')
plt.ylabel('Frequenza')
plt.show()

### 2.3 Matrice di Correlazione
La matrice di correlazione è un grafico che mostra le relazioni tra le variabili in un dataset. Permette di identificare facilmente le coppie di variabili che presentano una forte correlazione, sia positiva che negativa.

**Interpretazione:**

* **Colori:**
    * Rosso intenso: Correlazione positiva forte (le variabili aumentano o diminuiscono insieme).
    * Blu intenso: Correlazione negativa forte (le variabili aumentano in una direzione e diminuiscono nell'altra).
    * Bianco/grigio chiaro: Correlazione debole o assente (le variabili non mostrano una relazione lineare significativa).
* **Valori:** I numeri all'interno dei quadrati indicano il coefficiente di correlazione, che varia da -1 a 1.

**Cosa possiamo dedurre:**

* Identificare relazioni tra le variabili.
* Comprendere la direzione della relazione (positiva o negativa).
* Valutare l'intensità della relazione (forte o debole).

**Importante:**

* Correlazione non implica causazione.
* Interpretare nel contesto del dataset specifico.

**Osservazioni:**

* Alcune variabili sono fortemente correlate tra loro, come "mean radius" e "mean texture".
* Altre variabili non sono correlate tra loro, come "radius error" e "worst radius".





In [ ]:
plt.figure(figsize=(12, 10))
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', linewidths=0.2)
plt.title('Matrice di Correlazione')
plt.show()


### 2.4 Importanza delle Caratteristiche nel Dataset "Breast Cancer Wisconsin"

Questo grafico a barre rappresenta l'importanza delle caratteristiche nel dataset "Breast Cancer Wisconsin". L'importanza è stata calcolata utilizzando un algoritmo di Random Forest.

Le caratteristiche sono ordinate in base alla loro importanza, con la caratteristica più importante in alto. I valori di importanza sono rappresentati da barre colorate.

Dall'analisi del grafico è possibile osservare che alcune caratteristiche sono più importanti di altre per la classificazione dei tumori al seno. Quelle più importanti sono:

* **worst area**
* **worst concave points**
* **mean concave points**
* **worst radius**
* **worst perimeter**

Queste caratteristiche sono probabilmente quelle che forniscono più informazioni sulla natura del tumore e sulla sua probabilità di essere maligno.

L'analisi dell'importanza delle caratteristiche può essere utile per selezionare le caratteristiche più importanti da utilizzare in un modello di machine learning e per comprendere meglio i fattori che influenzano la classificazione dei tumori al seno.

Nei grafici successivi andremo ad analizzare queste 5 feature più importanti.


In [ ]:
# Dividere i dati in caratteristiche (X) e target (y)
X = df.drop(columns=['target'])
y = df['target']

# Addestrare un modello di RandomForest
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

# Ottenere l'importanza delle caratteristiche
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Visualizzare l'importanza delle caratteristiche
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df, palette='viridis', hue='Feature', dodge=False, legend=False)
plt.title('Importanza delle Caratteristiche')
plt.show()


In [ ]:
#Creo una lista con le prime 5 feature più importanti
topFeaturesByImportance = feature_importance_df[:5]['Feature'].values
print(topFeaturesByImportance)


### 2.5 Distribuzione delle Caratteristiche più Importanti nel Dataset "Breast Cancer Wisconsin"

Questo set di istogrammi mostra la distribuzione delle cinque caratteristiche più importanti nel dataset "Breast Cancer Wisconsin". Le caratteristiche sono state identificate utilizzando un algoritmo di Random Forest nel precedente passaggio.

Ogni istogramma mostra la distribuzione di una caratteristica per i tumori benigni (colore rosso) e maligni (colore blu). La densità di probabilità è rappresentata da una curva tratteggiata.

**Interpretazione dettagliata:**
* Per tutti i grafici possiamo notare come la distribuzione sia piuttosto marcata tra i due tipi di tumore, infatti queste cinque feature non a caso sono considerate le più importanti, appunto per questa loro caratteristica, possiamo notare che a sinistra dei grafici è presente una concentrazione più marcata di tumori benigni, mentre a destra è presente una concentrazione più marcata di tumori maligni.
**Considerazioni generali:**

* L'analisi di questi istogrammi conferma che le **caratteristiche più importanti** per la classificazione dei tumori al seno sono quelle relative alla **forma e alla dimensione del tumore**. Queste caratteristiche presentano una distribuzione differenziata tra i tumori benigni e maligni, suggerendo che possono essere utilizzate per distinguere tra i due tipi di tumore.
* L'analisi della distribuzione delle caratteristiche può essere utile per:
    * Comprendere meglio la relazione tra le caratteristiche del tumore e la sua natura (benigno o maligno).
    * Identificare potenziali biomarcatori per la diagnosi precoce del cancro al seno.
    * Sviluppare algoritmi di machine learning più efficaci per la classificazione dei tumori al seno.



In [ ]:
features = topFeaturesByImportance # Prendo le 5 feature più importanti

plt.figure(figsize=(20, 12))
for i, feature in enumerate(features):
    plt.subplot(2, 3, i + 1)
    sns.histplot(data=df, x=feature, hue='target', element='step', palette=['red', 'blue'], kde=True)
    plt.title(f'Distribuzione di {feature}')

plt.tight_layout()
plt.show()

### 2.6 Box Plot delle Caratteristiche più Importanti nel Dataset "Breast Cancer Wisconsin"

L'immagine mostra una serie di box plot che rappresentano la distribuzione di diverse caratteristiche per due diversi obiettivi ("target"). I box plot sono suddivisi in due righe e tre colonne. Ogni box plot rappresenta la distribuzione di una singola caratteristica per un singolo obiettivo.

**Elementi del box plot:**

* **Scatola:** La scatola rappresenta la distribuzione centrale dei dati. La linea centrale all'interno della scatola rappresenta la mediana.
* **Whisker:** I baffi si estendono dalla scatola fino al massimo e al minimo valore dei dati che non sono considerati outlier.
* **Outlier:** I punti dati che si trovano al di fuori dei baffi sono considerati outlier e sono rappresentati da punti singoli.

**Interpretazione dei box plot:**

* **Posizione della mediana:** La posizione della mediana all'interno della scatola indica la simmetria della distribuzione dei dati. Se la mediana è al centro della scatola, la distribuzione è simmetrica. Se la mediana è spostata verso un bordo della scatola, la distribuzione è asimmetrica.


In [ ]:
# Analisi esplorativa - Box Plot delle Caratteristiche
features = topFeaturesByImportance

plt.figure(figsize=(20, 12))
for i, feature in enumerate(features):
    plt.subplot(2, 3, i + 1)
    sns.boxplot(x='target', y=feature, data=df, palette=['red', 'blue'], hue='target', dodge=False, legend=False)
    plt.title(f'Box Plot di {feature}')
plt.tight_layout()
plt.show()

## 3. Eseguo l'algoritmo Decision Tree senza rumore

### 3.1 Split del dataset
Abbiamo scelto di dividere il dataset in training set (70%) e test set (30%).

In [ ]:
# Split del dataset in train e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Dimensione del training set (70%):", X_train.shape)
print("Dimensione del test set (30%):", X_test.shape)

### 3.2 Distribuzione dei target positivi e negativi nei set di training e test
Nei grafici sopra riportati, possiamo osservare la distribuzione delle classi del target nei set di training e test. 

- **Training Set**: 
  - Classe `1`: Rappresenta il 37.4% dei dati del training set.
  - Classe `0`: Rappresenta il 62.6% dei dati del training set.

- **Test Set**: 
  - Classe `1`: Rappresenta il 36.8% dei dati del test set.
  - Classe `0`: Rappresenta il 63.2% dei dati del test set.

La distribuzione delle classi è simile tra i due set, con una leggera predominanza della classe `0` in entrambi. Questo indica che il campionamento casuale effettuato con `train_test_split` ha mantenuto una distribuzione proporzionale delle classi tra training e test set, garantendo una buona rappresentatività dei dati nel processo di validazione del modello.


In [ ]:
# Conta le occorrenze di ogni classe nel training set e nel test set
train_counts = y_train.value_counts()
test_counts = y_test.value_counts()

# Calcola le percentuali
train_percentages = (train_counts / train_counts.sum()) * 100
test_percentages = (test_counts / test_counts.sum()) * 100

# Crea una figura e assi per i due grafici
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Grafico per il training set
train_counts.plot(kind='bar', ax=ax[0])
for i, count in enumerate(train_counts):
    ax[0].text(i, count, f'{train_percentages[i]:.1f}%', ha='center', va='bottom')
ax[0].set_title('Distribuzione del Target nel Training Set')
ax[0].set_xlabel('Classi')
ax[0].set_ylabel('Frequenza')

# Grafico per il test set
test_counts.plot(kind='bar', ax=ax[1])
for i, count in enumerate(test_counts):
    ax[1].text(i, count, f'{test_percentages[i]:.1f}%', ha='center', va='bottom')
ax[1].set_title('Distribuzione del Target nel Test Set')
ax[1].set_xlabel('Classi')
ax[1].set_ylabel('Frequenza')

# Mostra i grafici
plt.tight_layout()
plt.show()


### 3.2 Modello Decision tree

#### 3.2.1 Addestramento del modello Decision tree

In [ ]:
# Creiamo e addestriamo il modello di Decision Tree
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

#### 3.2.2 Valutazione del modello SVM

In [ ]:
#Avvio cronometro
start_time = time.time()

# Facciamo previsioni sul test set
y_pred = clf.predict(X_test)

#Stoppo cronometro
end_time = time.time()

# Valutiamo le performance del modello
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Calcola e stampa il classification report
class_report = classification_report(y_test, y_pred)

# Calcola la matrice di confusione
conf_matrix = confusion_matrix(y_test, y_pred)

elapsed_time_notModifiedDataset = end_time - start_time
print(f"Tempo trascorso: {elapsed_time_notModifiedDataset:.4f} secondi")


### 3.3 Valutazione della confusion matrix

In [ ]:
# Visualizza la matrice di confusione con seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

### 3.4 Classification report

In [ ]:
print("Classification Report:")
print(class_report)

## 4. Aggiunta di rumore al dataset

### 4.1 Metodo introduzione dati mancanti

In [ ]:
def introduce_missing_data_custom(dataset, columns, percentage):
    """
    Introduce valori nulli in un dataset specificando le colonne e la percentuale di dati da annullare.

    :param dataset: DataFrame su cui operare.
    :param columns: Lista delle colonne su cui introdurre i valori nulli.
    :param percentage: Percentuale dei dati da annullare (0-1).
    :return: DataFrame modificato.
    """
    # Calcola il numero di righe da annullare in base alla percentuale
    num_rows_to_null = int(len(dataset) * percentage)

    # Seleziona casualmente le righe da annullare
    rows_to_null = np.random.choice(dataset.index, num_rows_to_null, replace=False)
    #print("Vado a sporcare queste righe: ", rows_to_null)
    
    # Imposta i valori a NaN per le colonne specificate nelle righe selezionate
    dataset.loc[rows_to_null, columns] = np.nan

    return dataset

### 4.2 Metodo introduzione outliers

In [ ]:
def introduce_outliers(dataset, columns, percentage):
    """
    Introduce outliers in un dataset specificando le colonne e la percentuale di dati da modificare.

    :param dataset: DataFrame su cui operare.
    :param columns: Lista delle colonne su cui introdurre gli outliers.
    :param percentage: Percentuale dei dati da modificare (0-1).
    :return: DataFrame modificato.
    """
    # Calcola il numero di righe da modificare in base alla percentuale
    num_rows_to_change = int(len(dataset) * percentage)

    # Seleziona casualmente le righe da modificare
    rows_to_change = np.random.choice(dataset.index, num_rows_to_change, replace=False)
    #print("Vado a modificare queste righe: ", rows_to_change)

    # Per ogni riga selezionata, aggiungi un outlier alle colonne specificate
    for idx in rows_to_change:
        for column in columns:
            # Calcola un outlier come media + 5 * deviazione standard della colonna
            outlier_value = dataset[column].mean() + 5 * dataset[column].std()
            dataset.at[idx, column] = outlier_value

    return dataset

### 4.3 Metodo introduzione righe duplicate

In [ ]:
def introduce_duplicate_rows(dataset, percentage, y_train_spor):
    """
    Introduce righe duplicate in un dataset specificando la percentuale di righe da duplicare.
    Duplica anche le etichette corrispondenti in y_train_sporcata.

    :param dataset: DataFrame su cui operare.
    :param percentage: Percentuale di righe da duplicare (0-1).
    :param y_train_sporcata: Serie delle etichette corrispondenti.
    :return: DataFrame e Serie modificati.
    """
    # Calcola il numero di righe da duplicare in base alla percentuale
    num_rows_to_duplicate = int(len(dataset) * percentage)

    # Seleziona casualmente le righe da duplicare
    rows_to_duplicate = np.random.choice(dataset.index, num_rows_to_duplicate, replace=False)

    # Duplica le righe selezionate
    duplicated_rows = dataset.loc[rows_to_duplicate]
    dataset = pd.concat([dataset, duplicated_rows], ignore_index=True)

    # Duplica le etichette corrispondenti
    duplicated_labels = y_train_spor.loc[rows_to_duplicate]
    y_train_spor = pd.concat([y_train_spor, duplicated_labels], ignore_index=True)

    return dataset, y_train_spor

In [ ]:
def plot_feature_distribution_comparison(clean_df, noisy_df, feature_columns, top5Feature):
    """
    Confronta graficamente la distribuzione delle feature tra il dataset pulito e quello sporcato.

    :param clean_df: DataFrame del dataset pulito.
    :param noisy_df: DataFrame del dataset sporcato.
    :param feature_columns: Colonne delle feature da confrontare.
    :param top5Feature: Lista delle top 5 feature da mostrare.
    """
    # Crea copie dei DataFrame per evitare di modificare gli originali
    clean_df_copy = clean_df.copy()
    noisy_df_copy = noisy_df.copy()

    # Aggiungi una colonna per indicare lo stato (pulito o sporcato)
    clean_df_copy['status'] = 'clean'
    noisy_df_copy['status'] = 'noisy'

    # Seleziona solo le colonne delle top 5 feature
    feature_columns_to_plot = [col for col in feature_columns if col in top5Feature]

    # Concatena i due dataset
    combined_df = pd.concat([clean_df_copy, noisy_df_copy], ignore_index=True)

    # Calcola il numero di righe e colonne dei subplot in base al numero di feature da plottare
    num_features = len(feature_columns_to_plot)
    num_rows = (num_features + 2) // 3  # Calcola il numero di righe necessarie (3 colonne per riga)

    # Plotting
    plt.figure(figsize=(15, 5 * num_rows))  # Altezza ridotta per ridurre la dimensione complessiva del grafico
    sns.set(style="whitegrid")

    for i, column in enumerate(feature_columns_to_plot, 1):
        plt.subplot(num_rows, 3, i)
        sns.boxplot(x='status', y=column, data=combined_df)
        plt.title(column)
        plt.tight_layout()

    plt.suptitle('Confronto tra Dataset Pulito e Sporcato', y=1.02, fontsize=16, ha='center')
    plt.show()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
def plot_confusion_matrix(model):
    cm = confusion_matrix(y_test , y_pred_prob[: , 1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
    disp.plot(cmap=plt.cm.Blues)
    plt.show()

## 5. Esecuzione dell'algoritmo Decision Tree con rumore dinamico dal 10% al 100%
Il rumore viene introdotto sul dataset X_test, che abbiamo creato in precedenza e solo sulle top 5 colonne più importanti che vengono calcolate ad inizio notebook nel paragrafo 2.4.

In [ ]:
percentuali_rumore = [0.01,0.02,0.03,0.5,1]

### 5.1 Esecuzione Decision Tree con aggiunta di valori nulli

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc, accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt

# Aggiungi le metriche ROC al dizionario delle metriche
metricsNUll = {
    'percentuale': [],
    'accuracy': [],
    'f1_score': [],
    'precision': [],
    'recall': [],
    'fpr': [],   # False Positive Rate
    'tpr': [],   # True Positive Rate
    'roc_auc': [] # Area Under the Curve
}
datasetSporcatoNULL = {'percentuale': [], 'dataset': []}

for percentuale in percentuali_rumore:
    dataset_sporcato = introduce_missing_data_custom(X_train.copy(), topFeaturesByImportance, percentuale)
    
    datasetSporcatoNULL['dataset'].append(dataset_sporcato)
    datasetSporcatoNULL['percentuale'].append(percentuale * 100)
    
    # Creiamo e addestriamo il modello di Decision Tree
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(dataset_sporcato, y_train)

    # Effettua le previsioni sui dati di test
    y_pred_prob = clf.predict_proba(X_test)  # Probabilità predette
    y_pred = clf.predict(X_test)

    # Calcola le metriche di performance
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    # Calcola le curve ROC
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob[:, 1])  # Assumendo binario e la classe positiva è la seconda
    roc_auc = auc(fpr, tpr)

    # Salva le metriche
    metricsNUll['percentuale'].append(percentuale)
    metricsNUll['accuracy'].append(accuracy)
    metricsNUll['f1_score'].append(f1)
    metricsNUll['precision'].append(precision)
    metricsNUll['recall'].append(recall)
    metricsNUll['fpr'].append(fpr)
    metricsNUll['tpr'].append(tpr)
    metricsNUll['roc_auc'].append(roc_auc)

    print(f'Percentuale di rumore: {percentuale}. Accuracy: {accuracy}, AUC: {roc_auc}')
    plot_confusion_matrix(clf)
    
    print('K-FOLD: ----------------------------')


### 5.2 Esecuzione Decision Tree con aggiunta di outliers

In [ ]:
datasetSporcatoOutliers = {'percentuale': [], 'dataset': []}

# Aggiungi le metriche ROC al dizionario delle metriche
metricsOutliers = {
    'percentuale': [],
    'accuracy': [],
    'f1_score': [],
    'precision': [],
    'recall': [],
    'fpr': [],   # False Positive Rate
    'tpr': [],   # True Positive Rate
    'roc_auc': [] # Area Under the Curve
}

for percentuale in percentuali_rumore:
    dataset_sporcato = introduce_outliers(X_train.copy(), topFeaturesByImportance, percentuale)  # Funzione che introduce rumore nel dataset

    datasetSporcatoOutliers['dataset'].append(dataset_sporcato)
    datasetSporcatoOutliers['percentuale'].append(percentuale*100)

    # Creiamo e addestriamo il modello di Decision Tree con il dataset sporco
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(dataset_sporcato, y_train)

    # Effettua le previsioni sui dati di test
    y_pred_prob = clf.predict_proba(X_test)  # Probabilità predette
    y_pred = clf.predict(X_test)

    # Calcola le metriche di performance
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    # Calcola le curve ROC
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob[:, 1])  # Assumendo binario e la classe positiva è la seconda
    roc_auc = auc(fpr, tpr)

    # Salva le metriche
    metricsOutliers['percentuale'].append(percentuale)
    metricsOutliers['accuracy'].append(accuracy)
    metricsOutliers['f1_score'].append(f1)
    metricsOutliers['precision'].append(precision)
    metricsOutliers['recall'].append(recall)
    metricsOutliers['fpr'].append(fpr)
    metricsOutliers['tpr'].append(tpr)
    metricsOutliers['roc_auc'].append(roc_auc)

    print(f'Percentuale di rumore: {percentuale}. Accuracy: {accuracy}')
    plot_confusion_matrix(clf)    

    # Chiamata alla funzione di plot per confrontare le distribuzioni delle feature
    plot_feature_distribution_comparison(X_train, dataset_sporcato, X.columns, topFeaturesByImportance)

### 5.3 Esecuzione Decision Tree con aggiunta di righe duplicate

In [ ]:
datasetSporcatoDuplicate = {'percentuale': [], 'dataset': []}

metricsDuplicate = {
    'percentuale': [],
    'accuracy': [],
    'f1_score': [],
    'precision': [],
    'recall': [],
    'fpr': [],   # False Positive Rate
    'tpr': [],   # True Positive Rate
    'roc_auc': [] # Area Under the Curve
}

for percentuale in percentuali_rumore:
    dataset_sporcato, y_train_sporcata = introduce_duplicate_rows(X_train.copy(), percentuale, y_train.copy())  # Funzione che introduce rumore nel dataset

    datasetSporcatoDuplicate['dataset'].append(dataset_sporcato)
    datasetSporcatoDuplicate['percentuale'].append(percentuale*100)

    # Creiamo e addestriamo il modello di Decision Tree con il dataset sporco
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(dataset_sporcato, y_train_sporcata)

    # Effettua le previsioni sui dati di test
    y_pred_prob = clf.predict_proba(X_test)  # Probabilità predette
    y_pred = clf.predict(X_test)

    # Calcola le metriche di performance
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    # Calcola le curve ROC
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob[:, 1])  # Assumendo binario e la classe positiva è la seconda
    roc_auc = auc(fpr, tpr)

    # Salva le metriche
    metricsDuplicate['percentuale'].append(percentuale)
    metricsDuplicate['accuracy'].append(accuracy)
    metricsDuplicate['f1_score'].append(f1)
    metricsDuplicate['precision'].append(precision)
    metricsDuplicate['recall'].append(recall)
    metricsDuplicate['fpr'].append(fpr)
    metricsDuplicate['tpr'].append(tpr)
    metricsDuplicate['roc_auc'].append(roc_auc)

    print(f'Percentuale di rumore: {percentuale}. Accuracy: {accuracy}')

    # Chiamata alla funzione di plot per confrontare le distribuzioni delle feature
    plot_confusion_matrix(clf)
    plot_feature_distribution_comparison(X_train, dataset_sporcato, X.columns, topFeaturesByImportance)

## 6. Analisi dell'impatto del rumore su feature specifiche del dataset
In questa sezione andremo ad analizzare quanto impatta sul dataset la presenza di rumore su una feature. In particolare lo andremo a studiare singolarmente su quelle più importanti (prima l'aggiunta del rumore avvenita su tutte le top 5 feature, mentre adesso andiamo ad aggiungerlo a feature specifiche)

### 6.1 Definizione delle funzioni per l'aggiunta di rumore in una feature specifica
Vengono utilizzate le stesse funzioni create precedentemente per l'aggiunta di rumore


In [ ]:
def addestro_e_genero_plot(FEATURE_TO_EVALUATE, percentuali_rumore):
    num_percentuali = len(percentuali_rumore)

    # Configura la grandezza della figura in base al numero di percentuali
    if num_percentuali == 1:
        fig_width, fig_height = 12, 8  # Dimensioni più grandi per un singolo grafico
        font_size = 14
        rows = 1
        cols = 1
    else:
        cols = 3  # Numero di colonne nel layout
        rows = (num_percentuali + cols - 1) // cols  # Calcola il numero di righe necessarie
        fig_width, fig_height = cols * 5, rows * 4
        font_size = 10

    plt.figure(figsize=(fig_width, fig_height))  # Dimensione della figura

    for i, percentuale in enumerate(percentuali_rumore):
        # Introduci i dati mancanti
        dataset_sporcato = introduce_missing_data_custom(X_train.copy(), FEATURE_TO_EVALUATE, percentuale)

        # Allena il modello
        model = DecisionTreeClassifier(random_state=42)
        model.fit(dataset_sporcato, y_train)

        # Calcola l'accuratezza sui dati di test
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Ottieni l'importanza delle caratteristiche
        importances = model.feature_importances_
        feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
        feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

        # Evidenzia le caratteristiche sporcate
        feature_importance_df['Highlight'] = feature_importance_df['Feature'].isin(FEATURE_TO_EVALUATE)

        # Crea una sottotrama per ogni percentuale
        plt.subplot(rows, cols, i + 1)

        # Usa `hue` per colorare le barre e rimuovi la legenda automatica
        sns.barplot(x='Importance', y='Feature', data=feature_importance_df, hue='Highlight', palette={False: '#1f77b4', True: '#ffeb3b'}, dodge=False, legend=False)

        # Aggiungi l'accuratezza al titolo del grafico
        plt.title(f'Accuratezza: {accuracy:.2f}\nPercentuale: {percentuale}%', fontsize=font_size)
        plt.xlabel('Importanza', fontsize=font_size)
        plt.ylabel('Caratteristica', fontsize=font_size)
        plt.xticks(fontsize=font_size)
        plt.yticks(fontsize=font_size)

    plt.tight_layout()  # Ottimizza il layout
    plt.show()

### 6.2 Analisi del modello con le feature sporcate di valori NULLI

#### 6.2.1 Base per il nostro test
In questa sezione viene mostrata la base per il nostro test, eseguendo l'algoritmo sul dataset senza rumore, per vedere quali feature vengono considerate più importanti.

In [ ]:
FEATURE_TO_EVALUATE = [] # Lista vuota per non aggiungere rumore
percentuali_rumore = [0] # 0% per non aggiungere rumore

addestro_e_genero_plot(FEATURE_TO_EVALUATE, percentuali_rumore)

#### 6.2.2 Aggiunta di rumore alla feature "radius_worst"
Possiamo notare dal grafico precedente che la feature "radius_worst" è la più importante. In questa sezione andiamo ad aggiungere rumore solo a questa feature per vedere come cambia l'importanza delle feature.

In [ ]:
FEATURE_TO_EVALUATE = ['radius_worst']
percentuali_rumore = [0.001, 0.002, 0.003, 0.006, 0.01, 1] 

addestro_e_genero_plot(FEATURE_TO_EVALUATE, percentuali_rumore)

Possiamo notare che all'aumentare della percentuale di rumore, l'importanza della feature "radius_worst" diminuisce. Questo è dovuto al fatto che il rumore aggiunto alla feature "radius_worst" rende il modello meno affidabile rispetto alla feature "radius_worst" stessa. Notiamo, inoltre, che l'importanza della feature perimeter_worst aumenta e l'accuratezza del modello aumenta di conseguenza, possiamo quindi ipotizzare che dallo 0.6% di rumore all'1% di rumore il modello ha deciso di non considerare più radius_worst come feature importante, ma di basarsi su perimeter_worst.
E' interessante anche notare l'aumento dell'accuratezza appena l'algoritmo non considera più radius_worst come feature importante.

#### 6.2.3 Aggiunta di rumore alla feature "perimeter_worst" e "radius_worst"

In [ ]:
FEATURE_TO_EVALUATE = ['radius_worst', 'perimeter_worst']
percentuali_rumore = [0.003, 0.004, 0.005,  0.01,  0.02, 1]

addestro_e_genero_plot(FEATURE_TO_EVALUATE, percentuali_rumore)

Possiamo notare un comportamento simile a quello precedentemente studiato, ma con la differenza che a 100% di rumore su entrambe le feature l'algoritmo inizia a calare di accuratezza. Questo ci fa capire che l'algoritmo non è in grado di gestire un rumore così elevato su due feature importanti.
Andiamo adesso a studiare cosa succede se continuiamo a sporcare feature in un dataset

#### 6.2.4 Aggiunta di rumore su un numero elavato di feature

In [ ]:
FEATURE_TO_EVALUATE = ['radius_worst', 'perimeter_worst', 'area_worst', 'concave points_worst', 'concave points_mean', 'concavity_mean', 'area_mean', 'perimeter_mean', 'radius_mean', 'area_se']
percentuali_rumore = [0.1,  0.2, 0.3, 0.4, 0.5, 1]

addestro_e_genero_plot(FEATURE_TO_EVALUATE, percentuali_rumore)

Possiamo notare che nonostante il Binary tree sia un algoritmo molto robusto e consolidato per la classificazione con il numero di feature che abbiamo sporcato (pari a 10) l'accuratezza del modello è calata drasticamente per una qualsiasi percentuale di rumore al di sopra del 40%, cosa del tutto normale direi, ma ci fa capire l'importanza di un dataset pulito per l'addestramento di un modello.

### 6.3 Analisi del modello con le feature sporcate di OUTLIERS

#################################################################################################################

## 7. Conclusione e grafici riassuntivi

### 7.1 Grafico Decision Tree con valori nulli

In [ ]:
# Converti il dizionario in DataFrame
metrics_df = pd.DataFrame(metricsNUll)

# Plotting
plt.figure(figsize=(10, 6))
sns.set(style="darkgrid")
sns.lineplot(data=metrics_df, x='percentuale', y='accuracy', label='accuracy', marker='o')
sns.lineplot(data=metrics_df, x='percentuale', y='f1_score', label='f1Score', marker='o')
sns.lineplot(data=metrics_df, x='percentuale', y='precision', label='precision', marker='o')
sns.lineplot(data=metrics_df, x='percentuale', y='recall', label='recall', marker='o')

plt.title('Andamento dataset Breast Cancer Wisconsin con Decision Tree e valori nulli')
plt.xlabel('Percentuale di rumore')
plt.ylabel('Valore')
plt.legend()
plt.show()

### 7.2 Grafico Decision Tree con outliers

In [ ]:
# Converti il dizionario in DataFrame
metrics_df = pd.DataFrame(metricsOutliers)

# Plotting
plt.figure(figsize=(10, 6))
sns.set(style="darkgrid")
sns.lineplot(data=metrics_df, x='percentuale', y='accuracy', label='accuracy', marker='o')
sns.lineplot(data=metrics_df, x='percentuale', y='f1_score', label='f1Score', marker='o')
sns.lineplot(data=metrics_df, x='percentuale', y='precision', label='precision', marker='o')
sns.lineplot(data=metrics_df, x='percentuale', y='recall', label='recall', marker='o')

plt.title('Andamento dataset Breast Cancer Wisconsin con Decision Tree e outliers')
plt.xlabel('Percentuale di rumore')
plt.ylabel('Valore')
plt.legend()
plt.show()

### 7.3 Grafico Decision Tree con righe duplicate

In [ ]:
# Converti il dizionario in DataFrame
metrics_df = pd.DataFrame(metricsDuplicate)

# Plotting
plt.figure(figsize=(10, 6))
sns.set(style="darkgrid")
sns.lineplot(data=metrics_df, x='percentuale', y='accuracy', label='accuracy', marker='o')
sns.lineplot(data=metrics_df, x='percentuale', y='f1_score', label='f1Score', marker='o')
sns.lineplot(data=metrics_df, x='percentuale', y='precision', label='precision', marker='o')
sns.lineplot(data=metrics_df, x='percentuale', y='recall', label='recall', marker='o')

plt.title('Andamento dataset Breast Cancer Wisconsin con Decision Tree e valori duplicati')
plt.xlabel('Percentuale di rumore')
plt.ylabel('Valore')
plt.legend()
plt.show()

### 7.4 Curva ROC

In [ ]:
# Crea una figura con 3 subplot
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Funzione per tracciare la curva ROC
def plot_roc_curve(ax, metrics, title):
    for i, percentuale in enumerate(metrics['percentuale']):
        ax.plot(metrics['fpr'][i], metrics['tpr'][i], label=f'ROC curve (noise {percentuale}%) (AUC = {metrics["roc_auc"][i]:.2f})')
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(title)
    ax.legend(loc='lower right')
    ax.grid(True)

# Plot delle curve ROC per ciascuna condizione
plot_roc_curve(axs[0], metricsNUll, 'ROC for Null Data')
plot_roc_curve(axs[1], metricsOutliers, 'ROC for Outliers')
plot_roc_curve(axs[2], metricsDuplicate, 'ROC for Duplicates')

# Mostra la figura
plt.tight_layout()
plt.show()